# Установка пакетов разработки

In [1]:
!pip install torch torchvision torchaudio
!pip install torch-geometric
!pip install networkx
!pip install matplotlib seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.2 MB/s eta 0:00:00


# Функция вычисления графовых характеристик

In [2]:
import torch
import networkx as nx
from torch_geometric.datasets import Planetoid, OGB_MAG, PPI
from torch_geometric.utils import to_networkx
from networkx.algorithms.cluster import clustering
from networkx.algorithms.centrality import degree_centrality, closeness_centrality, betweenness_centrality, eigenvector_centrality
from networkx.algorithms.smallworld import sigma

def compute_metrics(graph_data, to_undirected=True):
    """ Функция для вычисления характеристик графа"""
    # Конвертируем граф в формат NetworkX
    G = to_networkx(graph_data, to_undirected=to_undirected)

    # Степень вершины
    try:
      degree_dict = dict(G.degree())
      print(f"Degree: {degree_dict}")
    except Exception as err:
      print(f"Degree error: {str(err)}")
      degree_dict = None

    # Диаметр графа
    if to_undirected:
      diameter = nx.diameter(G) if nx.is_connected(G) else None
      print(f"Diameter: {diameter}")
    else:
      diameter = None
      print(f"Diameter not supported for directed graphs")


    # Кластерный коэффициент для вершин
    try:
      clustering_coeff = clustering(G)
      print(f"Clustering Coefficient: {clustering_coeff}")
    except Exception as err:
      print(f"Clustering Coefficient error: {str(err)}")
      clustering_coeff = None


    # Центральность по степени
    try:
      degree_cent = degree_centrality(G)
      print(f"Degree Centrality: {degree_cent}")
    except Exception as err:
      print(f"Degree Centrality error: {str(err)}")
      degree_cent = None


    # Центральность по близости
    try:
      closeness_cent = closeness_centrality(G)
      print(f"Closeness Centrality: {closeness_cent}")
    except Exception as err:
      print(f"Closeness Centrality error: {str(err)}")
      closeness_cent = None


    # Междуузловая центральность
    try:
      betweenness_cent = betweenness_centrality(G)
      print(f"Betweenness Centrality: {betweenness_cent}")
    except Exception as err:
      print(f"Betweenness Centrality error: {str(err)}")
      betweenness_cent = None


    # Собственная центральность
    try:
      eigenvector_cent = eigenvector_centrality(G)
      print(f"Eigenvector Centrality: {eigenvector_cent}")
    except Exception as err:
      print(f"Eigenvector Centrality error: {str(err)}")
      eigenvector_cent = None

    # Эксцентриситет
    try:
      eccentricity = dict(nx.eccentricity(G))
      print(f"Eccentricity: {eccentricity}")
    except Exception as err:
      print(f"Eccentricity error: {str(err)}")
      eccentricity = None


    # Количество треугольников
    try:
      triangles = nx.triangles(G)
      print(f"Triangle Count: {triangles}")
    except Exception as err:
      print(f"Triangle Count error: {str(err)}")
      triangles = None


    return {
        "degree": degree_dict,
        "diameter": diameter,
        "clustering_coeff": clustering_coeff,
        "degree_cent": degree_cent,
        "closeness_cent": closeness_cent,
        "betweenness_cent": betweenness_cent,
        "eigenvector_cent": eigenvector_cent,
        "eccentricity": eccentricity,
        "triangles": triangles
    }

# Загрузка датасетов

In [3]:
def load_cora():
    return Planetoid(root='./data/Cora', name='Cora')[0]

cora_graph = load_cora()

Processing...
Done!
Processing...
Done!
Extracting data/OGB/mag/raw/mag.zip
Processing...
Done!


# Метод визуализации распределений графовых характеристик

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_metric_distribution(metric_data, metric_name, dataset_name):
    """Функция для визуализации распределений"""
    plt.figure(figsize=(8, 6))
    sns.histplot(list(metric_data.values()), kde=True, bins=30)
    plt.title(f'Distribution of {metric_name} in {dataset_name}')
    plt.xlabel(f'{metric_name} Values')
    plt.ylabel('Frequency')
    plt.show()

def visualize_metrics(dataset_name, metrics):
    for metric_name, metric_data in metrics.items():
        # Для характеристик в виде словарей
        if isinstance(metric_data, dict):
            plot_metric_distribution(metric_data, metric_name, dataset_name)
        else:
            # Если метрика одно значение или была вычислена, просто выводим сообщение
            print(f"{metric_name}: {metric_data}")

# Сэмплирование CoraML